In [1]:
import cv2
import math
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import time
import os

In [2]:
DATA_PATH = os.path.join('data1')

actions = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y'])

noOfSequences = 100

for action in actions:
    os.makedirs(os.path.join(DATA_PATH, action))


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data1\\A'

In [18]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

counter = 0
img_count = 22
    
offset = 20
imgSize = 300

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    if hands: 
        hand = hands[0]
        x, y, w, h = hand['bbox']
        
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8)*255
        
        imgCrop = img[y-offset:y + h + offset, x-offset:x + w + offset]
        
        imgCropShape = imgCrop.shape
        
        aspectRatio = h / w
        
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize-wCal)/2)
            imgWhite[:, wGap: wGap+wCal] = imgResize
            
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize-hCal)/2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
        
        cv2.imshow('ImageCrop', imgCrop)
        cv2.imshow('ImageWhite', imgWhite)
        
    cv2.imshow('Image', img)
    key = cv2.waitKey(1)
    
    if key == ord('s'):
        counter+=1
        npy_path = os.path.join(DATA_PATH, actions[img_count])
        cv2.imwrite(f'{npy_path}/{counter}.jpg',imgWhite)
        print(counter)
        if counter == noOfSequences:
            img_count+=1
            counter = 0

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Set the path to your sign language dataset
data_dir = 'data1'
img_width, img_height = 224, 224
batch_size = 32

# Load and preprocess the image data
images_list = []
labels = []
classes = os.listdir(data_dir)

for i, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
        image_array = tf.keras.preprocessing.image.img_to_array(image)
        images_list.append(image_array)
        labels.append(i)

images = np.array(images_list)
labels = np.array(labels)


In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

In [9]:

# Data augmentation for training images (optional but helpful to increase data diversity)
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow(train_images, train_labels, batch_size=batch_size)

# Data normalization for testing images
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow(test_images, test_labels, batch_size=batch_size)

# Create the CNN model


In [10]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(train_generator, validation_data=test_generator, epochs=10)

Epoch 1/10
60/60 [==============================] - 127s 2s/step - loss: 3.2134 - accuracy: 0.0484 - val_loss: 3.1737 - val_accuracy: 0.0458
Epoch 2/10
60/60 [==============================] - 125s 2s/step - loss: 3.0328 - accuracy: 0.0870 - val_loss: 2.8835 - val_accuracy: 0.1021
Epoch 3/10
60/60 [==============================] - 124s 2s/step - loss: 2.9066 - accuracy: 0.0979 - val_loss: 2.7430 - val_accuracy: 0.2104
Epoch 4/10
60/60 [==============================] - 126s 2s/step - loss: 2.7964 - accuracy: 0.1250 - val_loss: 2.6407 - val_accuracy: 0.1708
Epoch 5/10
60/60 [==============================] - 131s 2s/step - loss: 2.6496 - accuracy: 0.1411 - val_loss: 2.3777 - val_accuracy: 0.1854
Epoch 6/10
60/60 [==============================] - 126s 2s/step - loss: 2.5726 - accuracy: 0.1651 - val_loss: 2.1162 - val_accuracy: 0.3354
Epoch 7/10
60/60 [==============================] - 123s 2s/step - loss: 2.4361 - accuracy: 0.1979 - val_loss: 2.1278 - val_accuracy: 0.2729
Epoch 8/10
60

In [13]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')

15/15 [==============================] - 6s 368ms/step - loss: 1.7839 - accuracy: 0.4417
Test accuracy: 0.44


In [14]:
model.save('model.h5')
np.savetxt('labels.txt', classes, fmt='%s')

In [15]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier('model.h5', 'labels.txt')

offset = 20
imgSize = 300

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands: 
        hand = hands[0]
        x, y, w, h = hand['bbox']
        
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8)*255
        
        imgCrop = img[y-offset:y + h + offset, x-offset:x + w + offset]
        
        imgCropShape = imgCrop.shape
        
        aspectRatio = h / w
        
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize-wCal)/2)
            imgWhite[:, wGap: wGap+wCal] = imgResize
           
            
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize-hCal)/2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            
        prediction, index = classifier.getPrediction(imgWhite, draw = False)
        print(f'Predicted Gesture: {actions[index]}')
        cv2.rectangle(imgOutput, (x-offset, y-offset-50), (x-offset+90, y-offset), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput,actions[index], (x, y-25), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset), (x+w+offset, y+h+offset), (255, 0, 255), 4)
        
    cv2.imshow('Image', imgOutput)
    cv2.waitKey(1)

AttributeError: 'NoneType' object has no attribute 'copy'